Шаг 1. Агрегация по визитам
Задача: преобразовать транзакции в визиты. На уровне визита суммируются количество и сумма по товарам

In [28]:
import pandas as pd
import numpy as np

def aggregate_client_daily_items(df):
    """
    将交易数据转换为访问级别的聚合数据
    
    参数:
    df: 原始交易DataFrame，包含列: tr_date, client, item, item_group, quantity, amount
    
    返回:
    aggregated_df: 聚合后的DataFrame，包含列: client, visit_date, item, item_group, quantity, amount
    """
    
    print("=" * 80)
    print("步骤1: 访问聚合 - 将交易转换为访问级别数据")
    print("=" * 80)
    
    
    # 2. 显示原始数据信息
    print(f"原始数据形状: {df.shape}")
    print(f"原始交易记录数: {len(df):,}")
    
    # 3. 转换日期格式（如果尚未转换）
    if not pd.api.types.is_datetime64_any_dtype(df['tr_date']):
        try:
            df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y', errors='coerce')
        except:
            df['tr_date'] = pd.to_datetime(df['tr_date'], errors='coerce')
    
    # 4. 按(client, tr_date, item, item_group)分组并汇总
    aggregated_df = df.groupby(['client', 'tr_date', 'item', 'item_group'], as_index=False).agg({
        'quantity': 'sum',
        'amount': 'sum'
    })
    
    # 重命名列以符合要求
    aggregated_df = aggregated_df.rename(columns={'tr_date': 'visit_date'})
    
    # 5. 按client, visit_date, item排序
    aggregated_df = aggregated_df.sort_values(['client', 'visit_date', 'item']).reset_index(drop=True)
    
    # 6. 显示聚合结果信息
    print(f"\n聚合完成!")
    print(f"聚合后数据形状: {aggregated_df.shape}")
    print(f"聚合后行数: {len(aggregated_df):,}")
    
    # 7. 显示聚合前后的对比
    print(f"\n聚合前后对比:")
    print(f"  原始交易记录数: {len(df):,}")
    print(f"  聚合后访问记录数: {len(aggregated_df):,}")
    print(f"  聚合比例: {len(aggregated_df)/len(df)*100:.1f}%")
    
    if len(aggregated_df) < len(df):
        print(f"  合并了 {len(df)-len(aggregated_df):,} 条重复记录")
    
    # 8. 只显示前10个结果
    print(f"\n聚合后的数据示例 (前10行):")
    print("-" * 80)
    print(aggregated_df.head(10).to_string(index=False))
    print("-" * 80)
    
    # 9. 基本统计信息
    print(f"\n基本统计信息:")
    print(f"  总销售数量: {aggregated_df['quantity'].sum():,}")
    print(f"  总销售金额: {aggregated_df['amount'].sum():,.2f}")
    
    print("\n" + "=" * 80)
    print("步骤1完成: 成功将交易数据转换为访问级别聚合数据")
    print("=" * 80)
    
    return aggregated_df

# 使用示例
if __name__ == "__main__":
    # 读取原始数据
    df = pd.read_csv("transactions.csv")
    
    # 执行聚合函数
    aggregated_data = aggregate_client_daily_items(df)

步骤1: 访问聚合 - 将交易转换为访问级别数据
原始数据形状: (1008688, 7)
原始交易记录数: 1,008,688

聚合完成!
聚合后数据形状: (1003083, 6)
聚合后行数: 1,003,083

聚合前后对比:
  原始交易记录数: 1,008,688
  聚合后访问记录数: 1,003,083
  聚合比例: 99.4%
  合并了 5,605 条重复记录

聚合后的数据示例 (前10行):
--------------------------------------------------------------------------------
   client visit_date     item                   item_group  quantity  amount
  client1 2018-01-22 sku10765                Лаки и краски         1      29
  client1 2018-01-22 sku13695                Стойматериалы         5    1535
  client1 2018-01-22 sku29083                Лаки и краски         2     310
  client1 2018-01-22  sku2954                Лаки и краски         1     399
 client10 2019-08-05  sku1893                  Инструменты         1      79
 client10 2019-08-05  sku5624                  Инструменты         1      79
 client10 2019-08-05  sku7053                  Инструменты         1    4599
client100 2019-05-08 sku22214 Оборудование для сада и дачи         1    7299
client100 201

Шаг 2. Расчет профиля клиента (период наблюдения)
Задача: на дату актуальности (конец периода наблюдения) рассчитать для каждого клиента его профиль с 
RFM и дополнительными признаками.

In [37]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ============================================
# 步骤2: 计算客户画像（优化版本，修正显示问题）
# ============================================

def calculate_client_profile_at_date_fast(visits_df, observation_end_date):
    """
    优化的客户画像计算函数（使用向量化操作，避免循环）
    """
    
    print("=" * 80)
    print("步骤2: 计算客户画像（优化版本）")
    print("=" * 80)
    
    # 记录开始时间
    start_time = pd.Timestamp.now()
    
    # 1. 将观察结束日期转换为datetime格式
    observation_end = pd.to_datetime(observation_end_date)
    print(f"观察期结束日期: {observation_end.date()}")
    
    # 检查数据日期范围
    if 'visit_date' in visits_df.columns:
        print(f"数据日期范围: {visits_df['visit_date'].min().date()} 到 {visits_df['visit_date'].max().date()}")
    
    print(f"开始计算时间: {start_time}")
    
    # 2. 严格筛选观察期结束日期之前的访问记录
    filtered_visits = visits_df[visits_df['visit_date'] < observation_end].copy()
    
    print(f"筛选前访问记录数: {len(visits_df):,}")
    print(f"筛选后访问记录数: {len(filtered_visits):,}")
    print(f"筛选比例: {len(filtered_visits)/len(visits_df)*100:.1f}%")
    
    if len(filtered_visits) == 0:
        print("错误: 观察期内没有访问记录!")
        return pd.DataFrame()
    
    # 3. 检查数据中的客户数量
    unique_clients = filtered_visits['client'].nunique()
    print(f"观察期内唯一客户数: {unique_clients:,}")
    
    # 4. 先计算每次访问的总金额和总数量
    print("正在计算每次访问的汇总...")
    visit_summary = filtered_visits.groupby(['client', 'visit_date'], as_index=False).agg({
        'amount': 'sum',
        'quantity': 'sum'
    })
    
    # 5. 使用向量化操作一次性计算所有客户的画像
    print("正在使用向量化方法计算客户画像...")
    
    # 按客户分组计算汇总统计
    client_stats = visit_summary.groupby('client').agg({
        'visit_date': ['max', 'nunique'],
        'amount': ['sum', 'last'],
        'quantity': 'sum'
    })
    
    # 扁平化列名
    client_stats.columns = ['last_visit_date', 'frequency', 
                           'monetary', 'last_visit_amount', 'total_quantity']
    
    # 重置索引
    client_stats = client_stats.reset_index()
    
    # 6. 计算其他特征
    print("正在计算其他特征...")
    
    # Recency: 最近访问时间
    client_stats['recency'] = (observation_end - client_stats['last_visit_date']).dt.days
    
    # 平均单次消费金额
    client_stats['avg_purchase_value'] = client_stats['monetary'] / client_stats['frequency']
    
    # 每次访问平均商品数量
    client_stats['avg_items_per_visit'] = client_stats['total_quantity'] / client_stats['frequency']
    
    # 7. 计算需要额外分组的特征
    print("正在计算唯一商品数和周末访问次数...")
    
    # 唯一商品总数
    unique_items = filtered_visits.groupby('client')['item'].nunique().reset_index()
    unique_items.columns = ['client', 'total_unique_items']
    
    # 周末访问次数
    weekend_mask = filtered_visits['visit_date'].dt.dayofweek >= 5
    weekend_visits = (filtered_visits[weekend_mask]
                     .groupby('client')['visit_date']
                     .nunique()
                     .reset_index())
    weekend_visits.columns = ['client', 'weekend_visits']
    
    # 8. 合并所有特征
    client_profiles = client_stats.merge(unique_items, on='client', how='left')
    client_profiles = client_profiles.merge(weekend_visits, on='client', how='left')
    
    # 填充缺失的周末访问次数为0
    client_profiles['weekend_visits'] = client_profiles['weekend_visits'].fillna(0).astype(int)
    
    # 9. 重新排序列顺序
    columns_order = [
        'client', 'recency', 'frequency', 'monetary',
        'last_visit_date', 'total_quantity', 'avg_purchase_value',
        'total_unique_items', 'avg_items_per_visit', 'weekend_visits',
        'last_visit_amount'
    ]
    client_profiles = client_profiles[columns_order]
    
    # 10. 排序 - 按自然排序（确保client1, client2, client3...的顺序）
    # 首先提取数字部分进行排序
    def extract_client_number(client_id):
        # 移除"client"前缀，提取数字
        if isinstance(client_id, str) and client_id.startswith('client'):
            try:
                return int(client_id[6:])  # 移除"client"前缀（6个字符）
            except ValueError:
                return float('inf')  # 无法转换为数字的排在最后
        return float('inf')
    
    # 创建排序键列
    client_profiles['client_sort_key'] = client_profiles['client'].apply(extract_client_number)
    client_profiles = client_profiles.sort_values('client_sort_key').reset_index(drop=True)
    client_profiles = client_profiles.drop('client_sort_key', axis=1)
    
    # 11. 输出结果
    end_time = pd.Timestamp.now()
    processing_time = (end_time - start_time).total_seconds()

    # 12. 显示正确的前10个客户（按数字顺序）
    print(f"\n客户画像示例 (前10个客户，按数字顺序):")
    print("=" * 120)
    
    # 使用pandas的显示选项来确保所有列都能正确显示
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 200)
    pd.set_option('display.float_format', '{:.2f}'.format)
    
    # 创建显示的DataFrame
    display_df = client_profiles.head(10).copy()
    
    # 格式化显示
    print(display_df.to_string(index=False))
    print("=" * 120)
    
    # 恢复pandas默认设置
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    
    return client_profiles

# ============================================
# 主程序
# ============================================

if __name__ == "__main__":
    print("正在读取数据...")
    
    # 尝试读取步骤1的结果
    try:
        aggregated_data = pd.read_csv("aggregated_visits.csv")
        aggregated_data['visit_date'] = pd.to_datetime(aggregated_data['visit_date'])
        print(f"已加载聚合数据: {len(aggregated_data):,}行")
        
        # 检查数据的日期范围
        print(f"数据日期范围: {aggregated_data['visit_date'].min().date()} 到 {aggregated_data['visit_date'].max().date()}")
    except FileNotFoundError:
        print("未找到聚合数据文件，将重新计算步骤1...")
        
        # 读取原始数据并执行步骤1
        original_data = pd.read_csv("transactions.csv")
        
        # 步骤1的简化版本
        def aggregate_client_daily_items(df):
            required_columns = ['tr_date', 'client', 'item', 'item_group', 'quantity', 'amount']
            if not all(col in df.columns for col in required_columns):
                raise ValueError("数据中缺少必要的列")
            
            # 转换日期格式
            df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y', errors='coerce')
            
            # 检查日期转换是否成功
            valid_dates = df['tr_date'].notnull().sum()
            print(f"成功转换日期: {valid_dates:,}/{len(df):,} ({valid_dates/len(df)*100:.1f}%)")
            
            # 聚合
            aggregated = df.groupby(['client', 'tr_date', 'item', 'item_group'], as_index=False).agg({
                'quantity': 'sum',
                'amount': 'sum'
            }).rename(columns={'tr_date': 'visit_date'})
            
            # 排序
            aggregated = aggregated.sort_values(['client', 'visit_date', 'item'])
            
            print(f"步骤1完成: {len(aggregated):,}行")
            print(f"日期范围: {aggregated['visit_date'].min().date()} 到 {aggregated['visit_date'].max().date()}")
            return aggregated
        
        aggregated_data = aggregate_client_daily_items(original_data)
        aggregated_data.to_csv("aggregated_visits.csv", index=False)
        print("步骤1结果已保存")
    
    # 设置观察期结束日期
    observation_end_date = '2019-09-01'
    
    print(f"\n开始计算客户画像（观察期结束日期: {observation_end_date})...")
    print(f"客户总数: {aggregated_data['client'].nunique():,}")
    
    # 使用优化版本计算客户画像
    client_profiles = calculate_client_profile_at_date_fast(aggregated_data, observation_end_date)
    
    # 保存结果
    if not client_profiles.empty:
        output_file = f"client_profiles_{observation_end_date}.csv"
        client_profiles.to_csv(output_file, index=False)
        print(f"\n客户画像已保存到: {output_file}")
        

正在读取数据...
已加载聚合数据: 1,003,083行
数据日期范围: 2017-09-01 到 2019-10-31

开始计算客户画像（观察期结束日期: 2019-09-01)...
客户总数: 42,746
步骤2: 计算客户画像（优化版本）
观察期结束日期: 2019-09-01
数据日期范围: 2017-09-01 到 2019-10-31
开始计算时间: 2025-12-12 20:22:27.667849
筛选前访问记录数: 1,003,083
筛选后访问记录数: 892,864
筛选比例: 89.0%
观察期内唯一客户数: 39,906
正在计算每次访问的汇总...
正在使用向量化方法计算客户画像...
正在计算其他特征...
正在计算唯一商品数和周末访问次数...

客户画像示例 (前10个客户，按数字顺序):
  client  recency  frequency  monetary last_visit_date  total_quantity  avg_purchase_value  total_unique_items  avg_items_per_visit  weekend_visits  last_visit_amount
 client1      587          1      2273      2018-01-22               9             2273.00                   4                 9.00               0               2273
 client2      369          1      2499      2018-08-28               1             2499.00                   1                 1.00               0               2499
 client3        1          1      2682      2019-08-31               4             2682.00                   4                 

Шаг 3. Разметка события (период результата)
Задача: определить, посещал ли клиент магазин в заданный период [result_start, result_end)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ============================================
# 步骤1: 访问聚合 - 将交易转换为访问级别数据
# ============================================

def aggregate_client_daily_items(df):
    """
    将交易数据转换为访问级别的聚合数据
    
    参数:
    df: 原始交易DataFrame，包含列: tr_date, client, item, item_group, quantity, amount
    
    返回:
    aggregated_df: 聚合后的DataFrame，包含列: client, visit_date, item, item_group, quantity, amount
    """
    
    print("=" * 80)
    print("步骤1: 访问聚合 - 将交易转换为访问级别数据")
    print("=" * 80)
    
    # 1. 检查必要的列是否存在
    required_columns = ['tr_date', 'client', 'item', 'item_group', 'quantity', 'amount']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        raise ValueError(f"数据中缺少必要的列: {missing_columns}")
    
    # 2. 显示原始数据信息
    print(f"原始数据形状: {df.shape}")
    print(f"原始交易记录数: {len(df):,}")
    
    # 3. 转换日期格式（如果尚未转换）
    if not pd.api.types.is_datetime64_any_dtype(df['tr_date']):
        try:
            df['tr_date'] = pd.to_datetime(df['tr_date'], format='%d.%m.%Y', errors='coerce')
        except:
            df['tr_date'] = pd.to_datetime(df['tr_date'], errors='coerce')
    
    # 4. 按(client, tr_date, item, item_group)分组并汇总
    aggregated_df = df.groupby(['client', 'tr_date', 'item', 'item_group'], as_index=False).agg({
        'quantity': 'sum',
        'amount': 'sum'
    })
    
    # 重命名列以符合要求
    aggregated_df = aggregated_df.rename(columns={'tr_date': 'visit_date'})
    
    # 5. 按client, visit_date, item排序
    aggregated_df = aggregated_df.sort_values(['client', 'visit_date', 'item']).reset_index(drop=True)
    
    # 6. 显示聚合结果信息
    print(f"\n聚合完成!")
    print(f"聚合后数据形状: {aggregated_df.shape}")
    print(f"聚合后行数: {len(aggregated_df):,}")
    
    # 7. 显示聚合前后的对比
    print(f"\n聚合前后对比:")
    print(f"  原始交易记录数: {len(df):,}")
    print(f"  聚合后访问记录数: {len(aggregated_df):,}")
    print(f"  聚合比例: {len(aggregated_df)/len(df)*100:.1f}%")
    
    if len(aggregated_df) < len(df):
        print(f"  合并了 {len(df)-len(aggregated_df):,} 条重复记录")
    
    # 8. 只显示前10个结果
    print(f"\n聚合后的数据示例 (前10行):")
    print("-" * 80)
    print(aggregated_df.head(10).to_string(index=False))
    print("-" * 80)
    
    # 9. 基本统计信息
    print(f"\n基本统计信息:")
    print(f"  总销售数量: {aggregated_df['quantity'].sum():,}")
    print(f"  总销售金额: {aggregated_df['amount'].sum():,.2f}")
    
    print("\n" + "=" * 80)
    print("步骤1完成: 成功将交易数据转换为访问级别聚合数据")
    print("=" * 80)
    
    return aggregated_df

# ============================================
# 步骤2: 计算客户画像（观察期）- 简化版本
# ============================================

def calculate_client_profile_at_date_fast(visits_df, observation_end_date):
    """
    计算客户在观察期结束时的画像（简化版本）
    
    参数:
    visits_df: 访问级别的DataFrame
    observation_end_date: 观察期结束日期，格式为字符串 'YYYY-MM-DD'
    
    返回:
    client_profiles: 包含客户画像的DataFrame
    """
    
    print("=" * 80)
    print("步骤2: 计算客户画像（观察期）- 简化版本")
    print("=" * 80)
    
    # 1. 将观察结束日期转换为datetime格式
    observation_end = pd.to_datetime(observation_end_date)
    print(f"观察期结束日期: {observation_end.date()}")
    
    # 2. 严格筛选观察期结束日期之前的访问记录
    filtered_visits = visits_df[visits_df['visit_date'] < observation_end].copy()
    
    print(f"筛选后访问记录数: {len(filtered_visits):,}")
    
    if len(filtered_visits) == 0:
        print("警告: 观察期内没有访问记录!")
        return pd.DataFrame()
    
    # 3. 计算每次访问的汇总
    visit_summary = filtered_visits.groupby(['client', 'visit_date'], as_index=False).agg({
        'amount': 'sum',
        'quantity': 'sum'
    })
    
    # 4. 按客户分组计算RFM特征
    client_stats = visit_summary.groupby('client').agg({
        'visit_date': ['max', 'nunique'],
        'amount': ['sum', 'last'],
        'quantity': 'sum'
    })
    
    # 扁平化列名
    client_stats.columns = ['last_visit_date', 'frequency', 
                           'monetary', 'last_visit_amount', 'total_quantity']
    client_stats = client_stats.reset_index()
    
    # 5. 计算其他特征
    client_stats['recency'] = (observation_end - client_stats['last_visit_date']).dt.days
    client_stats['avg_purchase_value'] = client_stats['monetary'] / client_stats['frequency']
    client_stats['avg_items_per_visit'] = client_stats['total_quantity'] / client_stats['frequency']
    
    # 6. 计算唯一商品数和周末访问次数
    unique_items = filtered_visits.groupby('client')['item'].nunique().reset_index()
    unique_items.columns = ['client', 'total_unique_items']
    
    weekend_mask = filtered_visits['visit_date'].dt.dayofweek >= 5
    weekend_visits = (filtered_visits[weekend_mask]
                     .groupby('client')['visit_date']
                     .nunique()
                     .reset_index())
    weekend_visits.columns = ['client', 'weekend_visits']
    
    # 7. 合并所有特征
    client_profiles = client_stats.merge(unique_items, on='client', how='left')
    client_profiles = client_profiles.merge(weekend_visits, on='client', how='left')
    client_profiles['weekend_visits'] = client_profiles['weekend_visits'].fillna(0).astype(int)
    
    # 8. 重新排序列顺序并排序
    columns_order = [
        'client', 'recency', 'frequency', 'monetary',
        'last_visit_date', 'total_quantity', 'avg_purchase_value',
        'total_unique_items', 'avg_items_per_visit', 'weekend_visits',
        'last_visit_amount'
    ]
    client_profiles = client_profiles[columns_order]
    
    # 按client自然排序
    def extract_client_number(client_id):
        if isinstance(client_id, str) and client_id.startswith('client'):
            try:
                return int(client_id[6:])
            except ValueError:
                return float('inf')
        return float('inf')
    
    client_profiles['client_sort_key'] = client_profiles['client'].apply(extract_client_number)
    client_profiles = client_profiles.sort_values('client_sort_key').reset_index(drop=True)
    client_profiles = client_profiles.drop('client_sort_key', axis=1)
    
    # 9. 输出结果
    print(f"\n客户画像计算完成!")
    print(f"总客户数: {len(client_profiles):,}")
    
    print(f"\n客户画像示例 (前10个客户):")
    print("-" * 120)
    print(client_profiles.head(10).to_string(index=False))
    print("-" * 120)
    
    print("\n" + "=" * 80)
    print("步骤2完成: 成功计算客户画像")
    print("=" * 80)
    
    return client_profiles

# ============================================
# 步骤3: 事件标记（结果周期）
# ============================================

def mark_events(visits_df, result_start_date, result_end_date):
    """
    确定客户是否在给定周期内访问过商店
    
    参数:
    visits_df: 访问级别的DataFrame，包含列: client, visit_date
    result_start_date: 结果周期开始日期，格式为字符串 'YYYY-MM-DD'
    result_end_date: 结果周期结束日期，格式为字符串 'YYYY-MM-DD'
    
    返回:
    events_df: 包含两列的DataFrame: client, event (True/False)
    """
    
    print("=" * 80)
    print("步骤3: 事件标记（结果周期）")
    print("=" * 80)
    
    # 1. 将日期转换为datetime格式
    result_start = pd.to_datetime(result_start_date)
    result_end = pd.to_datetime(result_end_date)
    
    print(f"结果周期: {result_start.date()} 到 {result_end.date()}")
    print(f"周期长度: {(result_end - result_start).days} 天")
    
    # 2. 获取所有客户的唯一列表
    all_clients = visits_df['client'].unique()
    print(f"\n唯一客户总数: {len(all_clients):,}")
    
    # 3. 筛选严格范围内的访问记录
    # 条件: visit_date >= result_start_date AND visit_date < result_end_date
    result_period_visits = visits_df[
        (visits_df['visit_date'] >= result_start) & 
        (visits_df['visit_date'] < result_end)
    ].copy()
    
    print(f"结果周期内的访问记录数: {len(result_period_visits):,}")
    print(f"结果周期内访问的客户数: {result_period_visits['client'].nunique():,}")
    
    # 4. 确定哪些客户在该周期内至少访问过一次
    active_clients = result_period_visits['client'].unique()
    print(f"在结果周期内活跃的客户数: {len(active_clients):,}")
    
    # 5. 创建事件标记DataFrame
    events_df = pd.DataFrame({'client': all_clients})
    events_df['event'] = events_df['client'].isin(active_clients)
    
    # 6. 计算回复率
    response_rate = events_df['event'].mean() * 100
    print(f"\n事件标记统计:")
    print(f"  总客户数: {len(events_df):,}")
    print(f"  有事件的客户数: {events_df['event'].sum():,}")
    print(f"  回复率: {response_rate:.1f}% ({events_df['event'].sum():,}/{len(events_df):,})")
    
    # 7. 按客户自然排序
    def extract_client_number(client_id):
        if isinstance(client_id, str) and client_id.startswith('client'):
            try:
                return int(client_id[6:])
            except ValueError:
                return float('inf')
        return float('inf')
    
    events_df['client_sort_key'] = events_df['client'].apply(extract_client_number)
    events_df = events_df.sort_values('client_sort_key').reset_index(drop=True)
    events_df = events_df.drop('client_sort_key', axis=1)
    
    # 8. 显示结果示例
    print(f"\n事件标记示例 (前10个客户):")
    print("-" * 50)
    print(events_df.head(10).to_string(index=False))
    print("-" * 50)
    
    # 9. 按事件分组统计
    print(f"\n事件分布:")
    event_counts = events_df['event'].value_counts()
    for value, count in event_counts.items():
        percentage = count / len(events_df) * 100
        status = "有事件 (True)" if value else "无事件 (False)"
        print(f"  {status}: {count:,} 个客户 ({percentage:.1f}%)")
    
    # 10. 月度分析（如果数据支持）
    if len(result_period_visits) > 0:
        print(f"\n结果周期内访问的月度分布:")
        result_period_visits['month'] = result_period_visits['visit_date'].dt.to_period('M')
        monthly_counts = result_period_visits.groupby('month').agg({
            'client': 'nunique',
            'visit_date': 'count'
        }).reset_index()
        monthly_counts.columns = ['月份', '活跃客户数', '访问次数']
        monthly_counts['月份'] = monthly_counts['月份'].astype(str)
        print(monthly_counts.to_string(index=False))
    
    print("\n" + "=" * 80)
    print("步骤3完成: 成功标记事件")
    print("=" * 80)
    
    return events_df

# ============================================
# 主程序：执行所有步骤
# ============================================

if __name__ == "__main__":
    print("客户行为分析 - 完整流程")
    print("=" * 80)
    
    # 1. 读取原始交易数据
    print("正在读取原始交易数据...")
    try:
        original_data = pd.read_csv("transactions.csv")
        print(f"原始数据加载成功: {len(original_data):,}行")
    except FileNotFoundError:
        print("错误: 找不到transactions.csv文件")
        exit(1)
    
    # 2. 执行步骤1: 访问聚合
    print("\n" + "=" * 80)
    aggregated_data = aggregate_client_daily_items(original_data)
    
    # 保存步骤1结果
    aggregated_data.to_csv("aggregated_visits.csv", index=False)
    print("步骤1结果已保存到: aggregated_visits.csv")
    
    # 3. 执行步骤2: 客户画像（如果需要）
    print("\n" + "=" * 80)
    observation_end_date = '2019-09-01'  # 观察期结束日期
    aggregated_data['visit_date'] = pd.to_datetime(aggregated_data['visit_date'])
    
    client_profiles = calculate_client_profile_at_date_fast(aggregated_data, observation_end_date)
    
    # 保存步骤2结果
    if not client_profiles.empty:
        output_file = f"client_profiles_{observation_end_date}.csv"
        client_profiles.to_csv(output_file, index=False)
        print(f"步骤2结果已保存到: {output_file}")
    
    # 4. 执行步骤3: 事件标记
    print("\n" + "=" * 80)
    # 设置结果周期 [2019-09-01, 2019-10-01]
    result_start_date = '2019-09-01'
    result_end_date = '2019-10-01'
    
    events = mark_events(aggregated_data, result_start_date, result_end_date)
    
    # 保存步骤3结果
    events_output = f"events_{result_start_date}_to_{result_end_date}.csv"
    events.to_csv(events_output, index=False)
    print(f"步骤3结果已保存到: {events_output}")
    
    print("\n" + "=" * 80)
    print("所有步骤完成!")
    print("=" * 80)

客户行为分析 - 完整流程
正在读取原始交易数据...
原始数据加载成功: 1,008,688行

步骤1: 访问聚合 - 将交易转换为访问级别数据
原始数据形状: (1008688, 7)
原始交易记录数: 1,008,688

聚合完成!
聚合后数据形状: (1003083, 6)
聚合后行数: 1,003,083

聚合前后对比:
  原始交易记录数: 1,008,688
  聚合后访问记录数: 1,003,083
  聚合比例: 99.4%
  合并了 5,605 条重复记录

聚合后的数据示例 (前10行):
--------------------------------------------------------------------------------
   client visit_date     item                   item_group  quantity  amount
  client1 2018-01-22 sku10765                Лаки и краски         1      29
  client1 2018-01-22 sku13695                Стойматериалы         5    1535
  client1 2018-01-22 sku29083                Лаки и краски         2     310
  client1 2018-01-22  sku2954                Лаки и краски         1     399
 client10 2019-08-05  sku1893                  Инструменты         1      79
 client10 2019-08-05  sku5624                  Инструменты         1      79
 client10 2019-08-05  sku7053                  Инструменты         1    4599
client100 2019-05-08 sku22214 Оборудован

Шаг 4. Объединение профиля и события в выборку
Задача: соединить профили клиентов с разметкой события в единую выборку.